# SparkContext - number of workers and lazy evaluation

## Checking the impact of number of workers
While initializing the `SparkContext`, we can specify number of worker nodes. Generally, it is recommended to have one worker per core of the machine. But it can be smaller or larger. In the following code, we will examine the impact of number of worker cores on some parallelized operation.

In [1]:
from time import time
from pyspark import SparkContext

In [2]:
for j in range(1,5):
    sc= SparkContext(master = "local[%d]"%(j))
    t0=time()
    for i in range(10):
        sc.parallelize([1,2]*10000).reduce(lambda x,y:x+y)
    print(f"{j} executors, time = {time()-t0}")
    sc.stop()

1 executors, time = 1.6730988025665283
2 executors, time = 1.1461057662963867
3 executors, time = 1.0394089221954346
4 executors, time = 1.0736877918243408


#### We observe that it takes almost double time for 1 worker, and after that time reduces to a flat level for 2,3,4 workers etc. This is because this code run on a Linux virtual box using only 2 cores from the host machine. If you run this code on a machine with 4 cores, you will see benefit upto 4 cores and then the flattening out of the time taken. It also become clear that using more than one worker per core is not beneficial as it just does context-switching in that case and does not speed up the parallel computation.

## Showing the essence of _lazy_ evaluation
![](https://raw.githubusercontent.com/rubensa/spark-with-python/master/images/lazy.jpg)

In [3]:
sc = SparkContext(master="local[2]")

### Make a RDD with 1 million elements

In [4]:
%%time
rdd1 = sc.parallelize(range(1000000))

CPU times: user 0 ns, sys: 1.52 ms, total: 1.52 ms
Wall time: 3.9 ms


### Some computing function - `taketime`

In [5]:
from math import cos
def taketime(x):
    [cos(j) for j in range(100)]
    return cos(x)

### Check how much time is taken by `taketime` function

In [6]:
%%time
taketime(2)

CPU times: user 202 µs, sys: 66 µs, total: 268 µs
Wall time: 17.1 ms


-0.4161468365471424

### Now do the `map` operation on the function

In [7]:
%%time
interim = rdd1.map(lambda x: taketime(x))

CPU times: user 17 µs, sys: 6 µs, total: 23 µs
Wall time: 27.4 µs


#### How come each taketime function takes 45.8 us but the map operation with a 10000 element RDD also took similar time?<br><br>Because of _lazy_ evaluation i.e. nothing was computed in the previous step, just a plan of execution was made. The variable `interim` does not point to a data structure, instead it points to a plan of execution, expressed as a dependency graph. The dependency graph defines how RDDs are computed from each other.

### Let's see the "Dependency Graph" using `toDebugString` method

In [8]:
print(interim.toDebugString().decode())

(2) PythonRDD[1] at RDD at PythonRDD.scala:52 []
 |  ParallelCollectionRDD[0] at parallelize at PythonRDD.scala:194 []


![](https://raw.githubusercontent.com/rubensa/spark-with-python/master/images/rdd_dependency_graph.png)

### The actual execution by `reduce` method

In [9]:
%%time
print('output =',interim.reduce(lambda x,y:x+y))

output = -0.28870546796843666
CPU times: user 12.1 ms, sys: 0 ns, total: 12.1 ms
Wall time: 9.99 s


In [10]:
1000000*31e-6

31.0

#### It is less than what we would have expected considering 1 million operations with the `taketime` function. This is the result of parallel operation of 2 cores.

### Now, we have not saved (materialized) any intermediate results in `interim`, so another simple operation (e.g. counting elements > 0) will take almost same time

In [11]:
%%time
print(interim.filter(lambda x:x>0).count())

500000
CPU times: user 1.19 ms, sys: 9.74 ms, total: 10.9 ms
Wall time: 9.33 s


## Caching to reduce computation time on similar operation (spending memory)

### Run the same computation as before with `cache` method to tell the dependency graph to plan for caching

In [12]:
%%time
interim = rdd1.map(lambda x: taketime(x)).cache()

CPU times: user 7.04 ms, sys: 773 µs, total: 7.82 ms
Wall time: 17.8 ms


In [13]:
print(interim.toDebugString().decode())

(2) PythonRDD[4] at RDD at PythonRDD.scala:52 [Memory Serialized 1x Replicated]
 |  ParallelCollectionRDD[0] at parallelize at PythonRDD.scala:194 [Memory Serialized 1x Replicated]


In [14]:
%%time
print('output =',interim.reduce(lambda x,y:x+y))

output = -0.28870546796843666
CPU times: user 7.67 ms, sys: 0 ns, total: 7.67 ms
Wall time: 9.34 s


### Now run the same `filter` method with the help of cached result

In [15]:
%%time
print(interim.filter(lambda x:x>0).count())

500000
CPU times: user 7.07 ms, sys: 2.28 ms, total: 9.35 ms
Wall time: 317 ms


#### This time it took much shorter time due to cached result, which it could use to compare to 0 and count easily.